
### 필요 모듈 / 패키지

In [1]:
import pdfplumber
import pandas as pd
import os
import re
from collections import defaultdict
from unicodedata import normalize
from konlpy.tag import Okt
import openpyxl

## (1 -> 2) pdf파일을 텍스트화
* pdfplumber가 수능 시험지를 그대로 read 할 때, 왼쪽 -> 오른쪽이 아닌, 위에서 아래 순으로 읽는 문제가 생겼습니다.
* pdfplumber가 텍스트의 위치(좌표, x0, x1)를 제공해주고, 수능 시험지는 정확히 절반으로 나눠진다는 점을 이용하여, 순서대로 읽을 수 있게끔 분할하였습니다.

In [ ]:
for file in os.listdir("data/1_original"):
    print(f"current file: {file}")
    name_from = "../data/1_original/" + file
    name_to = "./data/2_texted" + file.replace(".pdf", ".txt")

    pdf_file = pdfplumber.open(name_from)
    w_file = open(name_to, "w", encoding="utf-8")

    # ① ~ ⑤ -> chr(9312) ~ chr(9316)
    # 한글 -> chr(44032 ~ 55203)
    # 한국용 한자 -> chr(19968 ~ 40959)

    for page in pdf_file.pages:
        print(f"{page} :", end=" ")
        left, right = [], []

        for word in page.extract_words():
            if word["x0"] < 400:
                target = left
            else:
                target = right

            if (re.match(r"\d{1,2}\.", word["text"]) # n.
                or re.match(r"\[\d{1,2}～\d{1,2}\]", word["text"]) # [m～n]
                or 9312 <= ord(word["text"][0]) <= 9316):  # ① ~ ⑤
                target.append("\n")

            target.append(word["text"])

        data = pd.DataFrame([*left, *right])

        if len(data):
            print("w /", end=" ")
            w_file.write(" ".join(data[0].tolist())+ "\n")
        else:
            print("n /", end=" ")

    pdf_file.close()
    w_file.close()
    print("")
print("done.")

### 한자 검증용 테스트 코드
* 한국에서 이용되는 한자는 모두 유니코드 19968 ~ 40959번 사이에 있습니다. (한국어문회 시험한자 특급 기준으로 검증)
* 간혹 같은 한자인데 다른 유니코드에 속한 문제가 있었는데, unicodedata모듈을 이용하여 이를 해결했습니다.
* 아래는 샘플을 취하여, 범위 밖에 있는 한자의 개수를 출력하는 코드입니다.

In [42]:
f = open("./hanja_translated/2023학년도-6월-모의평가-경제-문제.txt", "r", encoding="utf-8")

hanjas = defaultdict(int)

for letter in f.read():
    if 19968 <= ord(normalize("NFKC", letter)) <= 29973:
        hanjas[letter] += 1
f.close()
data = pd.DataFrame([[k for k in hanjas.keys()], [v for v in hanjas.values()]]).T
data.sort_values(by=1, ascending=False)

,0,1
76,國,70
116,材,33
59,個,31
52,價,31
114,年,31
...,...,...
142,慮,1
133,員,1
132,新,1
129,切,1


## (2 -> 3) 불필요한 부분 정제
* 비록 텍스트화 하였지만, 시험지 상단의 "홀수형", "2025년 수학능력시험...", "...저작권은 한국교육평가원..."등의 데이터 노이즈가 있었습니다.
* 코드화 시키기 어렵고, 사람이 직접하는 것이 시간을 아낄 수 있다고 판단, 직접 작업하여 정제했습니다.
* 이미 한자로 쓰여진 한자어는 3_2 폴더에 분리하였습니다.

## (3 -> 4) 한자어가 될 수 있는 단어 오른쪽에 괄호 표기
* 생성형 AI(ChatGPT, Gemini) 에게 바로 한자변환을 맡기고자 했으나, 프롬프트 요구사항이 많고, 정확도가 90% 내외정도가 되며, 한자변환이 진행될수록 정확도가 비약적으로 떨어지는 증상이 생겨, 바로 한자변환을 시키는데 어려움이 있었습니다.
* 이를 해결하기 위해, konlpy 모듈을 이용하여, 형태소 단위별로 쪼개 한자어일 가능성이 존재하는 품사에만 괄호를 치도록 하고, 해당 단어만 한자를 변환하도록 요청하여 작업시간을 어느 정도 줄이고, 정확도를 크게 높이는(97%) 결과를 가져왔습니다.

In [7]:
okt = Okt()
label = ["Noun", "Suffix", "Adjective",
         "Adverb", "Determiner", "Prefix"]      # 한자어가 존재할 가능성이 있는 품사 모음

for file_name in os.listdir("data/3_text_labeled"):
    print(f"current file: {file_name}")
    name_from = "./data/3_text_labeled/" + file_name
    name_to = "./data/4_word_found/" + file_name

    f = open(name_from, "r", encoding="utf-8")
    f2 = open(name_to, "w", encoding="utf-8")

    for text in f.readlines():
        for res in okt.pos(text):
            f2.write(res[0])

            if res[1] in label:
                f2.write("{" + res[0] + "}")
            f2.write(" ")
        f2.write("\n")

    f.close()
    f2.close()

current file: 2023학년도-6월-모의평가-국어-문제(수정).txt
current file: 2023학년도-9월-모의평가-국어-문제(수정).txt
current file: 2023학년도-대학수학능력시험-국어-문제(수정).txt
current file: 2024학년도-6월-모의평가-국어-문제(수정).txt
current file: 2024학년도-9월-모의평가-국어-문제(수정).txt
current file: 2024학년도-대학수학능력시험-국어-문제(수정).txt
current file: 2025학년도-6월-모의평가-국어-문제(수정).txt
current file: 2025학년도-9월-모의평가-국어-문제(수정).txt
current file: 2025학년도-대학수학능력시험-국어-문제(수정).txt
current file: 2026년-9월-고3-모의고사-국어-문제(수정).txt
current file: 2026학년도-6월-모의평가-국어-문제(수정).txt
current file: 2026학년도-대학수학능력시험-국어-문제(수정).txt
current file: 2026학년도-대학수학능력시험-사회문화-문제(수정).txt
current file: 2026학년도-대학수학능력시험-생활과윤리-문제(수정).txt
current file: 2026학년도-대학수학능력시험-세계사-문제(수정).txt
current file: 2026학년도-대학수학능력시험-세계지리-문제(수정).txt
current file: 2026학년도-대학수학능력시험-윤리와사상-문제(수정).txt
current file: 2026학년도-대학수학능력시험-정치와법-문제(수정).txt
current file: 2026학년도-대학수학능력시험-한국사-문제(수정).txt
current file: 2026학년도-대학수학능력시험-한국지리-문제(수정).txt


## 4 -> 5) 한자변환작업
* 한자어 변환 AI로 Gemini를 선택했는데, 시간상 무제한으로 사용할 수 있는 모델이 필요했고, 무제한으로 쓸 수 있는 Chatgpt-4.0o 모델과 비교하여 Gemini 빠른모드가 더 정확도가 높다는 점을 고려하였습니다. (나중에 4->5에 정리할 설명 부분)

## 5 -> 6) 데이터베이스화
* 만들어진 한자를 바탕으로, 데이터를 추출하였습니다.
* 데이터 하나의 구성은 `출제연도, 출제월, 과목, 지문번호, 문제번호, 한자` 순으로 구성되어 있습니다.
* 총 데이터의 수는 5만개+ 입니다.

In [2]:
hanjas = defaultdict(int)
hanja_data = []


for file_name in os.listdir("data/5_hanja_translated"):
    print(f"current file: {file_name}")
    file_elements = file_name.replace(".txt", "").split("-")    # 파일 이름 요소
    f = open("./data/5_hanja_translated/" + file_name, "r", encoding="utf-8")
    p_num, q_num = 1, 1     # 지문번호, 문제번호

    nz = ""
    flower = False
    for letter in f.read():
        # 꽃 내부 부분에 대한 처리
        if letter == "✿":
            flower = not flower
            if not flower:
                label = [*file_elements, p_num, q_num]
                nz = nz.split(",")
                if nz[0] == "지문":
                    p_num += 1
                elif nz[0] == "문제":
                    q_num += 1
                nz = ""
            continue

        if flower:
            nz += letter
            continue

        # 꽃 외부 부분에 대한 처리
        try:
            if 19968 <= ord(normalize("NFKC", letter)) <= 29973:
                hanja_data.append([*label, letter])
        except TypeError:
            continue
    f.close()


# pd.Series(hanjas)
data = pd.DataFrame(hanja_data,
                    columns=["출제연도", "출제월", "과목", "지문번호", "문제번호", "한자"])
data.to_csv("./data/6_csv/final_data.csv", sep=",", encoding="utf-8")
data.to_excel("./data/6_csv/final_data.xlsx", index=True)


current file: 2023-11-국어.txt
current file: 2023-6-경제.txt
current file: 2023-6-국어.txt
current file: 2023-6-동아시아.txt
current file: 2023-9-국어.txt
current file: 2026-11-국어.txt
current file: 2026-11-사회문화.txt
current file: 2026-11-생활과윤리.txt
current file: 2026-11-세계사.txt
current file: 2026-11-세계지리.txt
current file: 2026-11-윤리와사상.txt
current file: 2026-11-정치와법.txt
current file: 2026-11-한국사.txt
current file: 2026-11-한국지리.txt
current file: 2026-6-국어.txt
current file: 2026-9-국어.txt


## +) 교육기관별 상용한자 및 급수한자
* 8급 -> 80
* 준5급 -> 55
* 중등 -> 2
* 고등 -> 1
* 급외자나 특급, 사범 등, 1급을 넘는 범위 -> 0

#### 출처
* 한국어문회 : 나무위키
* 대한검정회 : 대한검정회 사이트
* 한문교육용기초한자 : 대한상공회 (8~3급) 사이트

In [24]:
path = "data/01_hanja_ref/"
hanja_rank = defaultdict(lambda : [0, 0, 0])    # ['대한검정회', '한국어문회', '한문교육용기초한자']
i = 0
for dir_name in sorted(os.listdir(path)):
    for file_name in os.listdir(path + dir_name):
        f = open(path + dir_name + "/" + file_name, "r", encoding="utf-8")
        for letter in f.read():
            letter = normalize("NFKC", letter)
            if 19968 <= ord(letter) <= 29973:
                hanja_rank[letter][i] = int(file_name.replace(".txt", ""))

    i += 1
h_df = pd.DataFrame(hanja_rank.keys(), columns=["한자"])
r_df = pd.DataFrame(hanja_rank.values(),
             columns=["등급(대한검정회)", "등급(한국어문회)", "등급(기초교육용)"]
             )

hr_df = pd.concat([h_df, r_df], axis=1)

# 값 수정
def f(x):
    if x == 1:
        return "고등"
    elif x == 2:
        return "중등"
    else:
        return "심화"

hr_df["등급(대한검정회)"] /= 10
hr_df["등급(한국어문회)"] /= 10
hr_df["등급(기초교육용)"] = hr_df["등급(기초교육용)"].apply(f)

# 저장
hr_df.to_excel("./data/02_ref_db/ref.xlsx", index=False)
hr_df.to_csv("./data/02_ref_db/ref.csv", sep=",", encoding="utf-8")

